In [1]:
import pymc3 as pm
import numpy as np

In [2]:
T = 1000
k = 5
d1 = 20
d2 = 30

In [3]:
W1 = np.random.randn(d1, k)
W2 = np.random.randn(d2, k)
mu1 = np.random.randn(d1, 1)
mu2 = np.random.randn(d2, 1)
pre_Psi1 = np.random.randn(2 * d1, d1)
Psi1 = np.dot(pre_Psi1.T, pre_Psi1)
pre_Psi2 = np.random.randn(2 * d2, d2)
Psi2 = np.dot(pre_Psi2.T, pre_Psi2)

In [4]:
Psi1_sqrt = np.linalg.cholesky(Psi1)
Psi2_sqrt = np.linalg.cholesky(Psi2)

In [5]:
Z = np.random.randn(T, k)
Z_lift1 = np.dot(W1, Z.T).T
Z_lift2 = np.dot(W2, Z.T).T
Z_shift1 = Z_lift1 + mu1.T
Z_shift2 = Z_lift2 + mu2.T
X1_noise = np.random.randn(T, d1)
X2_noise = np.random.randn(T, d2)
X1 = np.dot(X1_noise, Psi1_sqrt) + Z_shift1
X2 = np.dot(X2_noise, Psi2_sqrt) + Z_shift2

In [6]:
print(X1)
print(X2)

[[  3.33401584  10.64728792  13.82630138 ...,  -0.52312009  -2.14791984
   11.2942102 ]
 [ -1.43581971   1.09525442  17.10016658 ...,  -0.90589106  -1.73072043
    0.37660626]
 [  2.64919251  -9.86593874 -21.0143882  ...,   3.81349619  -5.49531794
  -11.62867467]
 ..., 
 [  0.37630666 -13.89471274   0.54882232 ...,  12.36101795   1.59105052
   -3.16888104]
 [-11.40718361  17.35840044  12.14152536 ...,   3.26909135  -1.08402167
    6.20531117]
 [  0.62768025  -9.33015414   8.82250742 ...,   0.55885336   1.47142527
    7.36688892]]
[[ -0.26442644  -9.96782253  22.63418662 ...,  -5.12436896   0.32786751
   11.09903507]
 [  1.90529576   9.43013045  -3.34520628 ...,   3.84116991  -1.0254626
  -10.05731376]
 [  0.85162623   0.09491229   6.95024424 ...,  -0.95951293  -0.18701062
    7.60312167]
 ..., 
 [-14.53892887  -4.54217791  -4.76362778 ...,  -6.22291993   1.21701747
   -3.15814095]
 [  5.95695681  15.20629375   0.28164217 ...,   8.26632552  -5.14262207
    2.82002565]
 [ -9.28056866  -2

In [7]:
model = pm.Model()

with model:
    # Priors on Z-lifting matrices
    W1_var = pm.Normal('W1', shape=(d1,k))
    W2_var = pm.Normal('W2', shape=(d2,k))
    
    # Priors on X means
    mu1_var = pm.Normal('mu1', shape=(d1,1))
    mu2_var = pm.Normal('mu2', shape=(d2,1))
    
    # Priors on covariance matrices
    Psi1_var = pm.LKJCholeskyCov(
        'Psi1', n=d1, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    Psi2_var = pm.LKJCholeskyCov(
        'Psi2', n=d2, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    
    # SDs for the Xs
    L1 = pm.expand_packed_triangular(d1, Psi1_var)
    L2 = pm.expand_packed_triangular(d2, Psi2_var)
    
    # Likelihood on Z
    Z_var = pm.Normal('Z', shape=(T,k))
    
    # Means for conditional likelihood on Xs conditioned on Z
    Z_lift1_var = W1_var.dot(Z_var.T).T
    Z_lift2_var = W2_var.dot(Z_var.T).T
    Z_shift1_var = Z_lift1_var + mu1_var.T
    Z_shift2_var = Z_lift2_var + mu2_var.T
    
    # Conditional likelihoods of Xs conditioned on Z
    X1_var = pm.MvNormal(name='X1', mu=Z_shift1_var, chol=L1, observed=X1)
    X2_var = pm.MvNormal(name='X2', mu=Z_shift2_var, chol=L2, observed=X2)
    
    pm.sample(live_plot)

In [11]:
map_estimate = pm.find_MAP(model=model, method='NUTS')

  0%|          | 0/5000 [00:00<?, ?it/s]


ValueError: Unknown solver NUTS

In [9]:
map_estimate.keys()

dict_keys(['Z', 'mu1', 'Psi2_cholesky_cov_packed__', 'mu2', 'Psi1_cholesky_cov_packed__', 'Psi2', 'W1', 'Psi1', 'W2'])

In [10]:
help(pm.find_MAP)

Help on function find_MAP in module pymc3.tuning.starting:

find_MAP(start=None, vars=None, method='L-BFGS-B', return_raw=False, include_transformed=True, progressbar=True, maxeval=5000, model=None, *args, **kwargs)
    Finds the local maximum a posteriori point given a model.
    
    Parameters
    ----------
    start : `dict` of parameter values (Defaults to `model.test_point`)
    vars : list
        List of variables to optimize and set to optimum (Defaults to all continuous).
    method : string or callable
        Optimization algorithm (Defaults to 'L-BFGS-B' unless
        discrete variables are specified in `vars`, then
        `Powell` which will perform better).  For instructions on use of a callable,
        refer to SciPy's documentation of `optimize.minimize`.
    return_raw : bool
        Whether to return the full output of scipy.optimize.minimize (Defaults to `False`)
    include_transformed : bool
        Flag for reporting automatically transformed variables in add